In [15]:
import requests
from bs4 import BeautifulSoup as bs
from  tqdm.notebook import tqdm
from time import sleep
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline

import time
import random
import re

In [7]:
main_link='https://www.captainbook.gr/bookstore/year/2017?page='

headers = {'User-agent': ''}

response = requests.get(main_link, headers=headers)

if response.status_code == 403:
    print('you got cought!')

In [5]:
soup = bs(response.content, 'html.parser')

In [6]:
for_p = soup.find_all('ul', 
                      class_='pagination justify-content-end')[0].find('li', 
                                                                        class_="page_item d-flex align-items-center")

max_pages = for_p.find_all('a')[-2].get_text()

In [9]:
price_old = []
price_new = []
price_discount = []

In [24]:
books = []
publisher = []
links = []

In [25]:
for page in tqdm(range(1,int(max_pages)+1)):
    response = requests.get(main_link+str(page), headers = headers)
    if response.status_code == 200:
        soup = bs(response.content, 'html.parser')
        titles = soup.find_all('h1')
        prices = soup.find_all('div', class_='price p-0 d-flex mb-2 align-items-center justify-content-between')
        publ = soup.find_all('h2', class_='p-0')
        for i in range(12):
            try: 
                price_old.append(prices[i].find_all('span', class_="old-price p-0")[0].get_text().strip())
            except:
                price_old.append(np.nan)
            try:
                price_new.append(prices[i].find_all('span', class_="new-price p-0")[0].get_text().strip())
            except:
                price_new.append(np.nan)
            try:
                price_discount.append(prices[i].find_all('small', class_="p-0")[0].get_text().strip())
            except:
                price_discount.append(np.nan)
            try:
                publisher.append(publ[i].get_text().strip())
            except:
                publisher.append(np.nan)
            try:
                books.append(titles[i].get_text().strip())
            except:
                books.append(np.nan)
            try:
                links.append(titles[i].find('a', class_="p-0 js-title productClickTracker").get('href'))
            except:
                links.append(np.nan)
    else:
        saved = i
        print('You got caught!')
        break
    
    time.sleep(random.uniform(0,3))   

  0%|          | 0/1098 [00:00<?, ?it/s]

In [26]:
dic = {'Title': books,
    'Price_Old': price_old,
    'Discount': price_discount,
    'Price_New': price_new,
    'Writer': publisher,
    'Link': links }

In [27]:
df = pd.DataFrame(dic, columns=['Title','Price_Old','Discount','Price_New','Publisher','Link'])

In [28]:
df['Year'] = 2017

In [29]:
df.isna().sum()

Title           6
Price_Old    2236
Discount     2236
Price_New    1738
Publisher       6
Link            6
Year            0
dtype: int64

In [30]:
df.sample(20)

,Title,Price_Old,Discount,Price_New,Publisher,Link,Year
9190,Γλώσσα με άρωμα_ Βόρειας Εύβοιας,17.50€,-11%,15.58€,Ασημίνα Ντέλιου,https://www.captainbook.gr/book/glossa-me-arom...,2017
3951,GENIAL KLICK A2 ARBEITSBUCH (+ 2…,20.35€,-16%,17.09€,JEFF KINNEY,https://www.captainbook.gr/book/genial-klick-a...,2017
9301,Θέλω να γίνω... Πυροσβέστης,5.50€,-26%,4.07€,,https://www.captainbook.gr/book/thelo-na-gino-...,2017
9216,Το κύμα και η θάλασσα,14.30€,-11%,12.73€,Σοφία Παπαηλιάδου,https://www.captainbook.gr/book/to-kuma-kai-i-...,2017
6322,Ένα πρόγραμμα Νέας Οικονομικής…,NaN,NaN,NaN,Θεόδωρος Μαριόλης,https://www.captainbook.gr/book/ena-programma-...,2017
11833,Ο ΔΙΚΟΣ ΜΑΣ ΠΑΪΣΙΟΣ,8.49€,-11%,7.56€,Συλλογικό έργο,https://www.captainbook.gr/book/o-dikos-mas-pa...,2017
5437,99%,12.00€,-11%,10.68€,Pierre Laurent,https://www.captainbook.gr/book/99-percent--97...,2017
2773,"Ο Μέγας Βασίλειος θεολογεί,…",NaN,NaN,5.00€,,https://www.captainbook.gr/book/o-megas-basile...,2017
10116,"Η Εκκλησία, ο παπισμός και το…",11.00€,-27%,8.03€,Philip Sherrard,https://www.captainbook.gr/book/i-ekklisia-o-p...,2017
2225,HERE COME THE PUPPIES: A…,11.11€,-11%,9.89€,A,https://www.captainbook.gr/book/here-come-the-...,2017


In [31]:
df.to_csv('../data/books_2017.csv')

In [2]:
df = pd.read_csv('../data/books_2017.csv')

In [4]:
df.drop('Unnamed: 0', axis=1,inplace=True)

In [5]:
df.head()

,Title,Price_Old,Discount,Price_New,Publisher,Link,Year
0,22 Οκτωβρίου 1717: Η λησμονημένη…,10.60€,-11%,9.43€,Δημοσθένης Δόνος,https://www.captainbook.gr/book/22-oktobriou-1...,2017
1,THE DUNE NOVELS MESSIAH,13.77€,-11%,12.26€,9781529347869,https://www.captainbook.gr/book/the-dune-novel...,2017
2,ΜΕΤΡΗΣΗ ΚΑΙ ΑΞΙΟΛΟΓΗΣΗ ΣΤΗΝ…,NaN,NaN,22.26€,ΚΩΝΣΤΑΝΤΙΝΟΣ ΠΑΠΑΝΑΣΤΑΣΙΟΥ,https://www.captainbook.gr/book/metrisi-kai-ak...,2017
3,Διεθνή Λογιστικά Πρότυπα,24.38€,-11%,21.70€,NaN,https://www.captainbook.gr/book/diethni-logist...,2017
4,Χαράλαμπος Μ. Μουτσόπουλος,14.00€,-21%,11.06€,Κωνσταντίνος Δ. Κόλλιος,https://www.captainbook.gr/book/xaralampos-m-m...,2017


In [20]:
category = []
ISBN = []
pages=[]
binding = []

In [21]:
for link in tqdm(df.Link):
    try:
        response = requests.get(link, headers = headers)
    
        if response.status_code == 200:
            soup1 = bs(response.content, 'html.parser')
            body = soup1.find('div', class_="fading col-md-12 p-0").find_all('div', class_="td")
            body1=soup1.find_all('li', class_="py-2 d-flex justify-content-between")
            
            try:
                count1=0
                count2=0
                count3=0
                count4=0

                for bod in body1:
                    if re.search(r'Κατηγορίες',str(bod)):
                        category.append(bod.get_text().strip().split('\n')[-1])
                        count1+=1
                    elif re.search(r'ISBN',str(bod)):
                        ISBN.append(bod.get_text().strip().split('\n')[-1])
                        count2+=1
                
                for i in range(len(body)):
                    if re.search(r'Σελίδες',str(body[i])):
                        pages.append(body[i+1].get_text().strip())
                        count3 +=1
                    elif re.search(r'Δέσιμο',str(body[i])):
                        binding.append(body[i+1].get_text().strip())
                        count4 +=1

                if count1 == 0:
                    category.append(np.nan)
                if count2 == 0:
                    ISBN.append(np.nan)
                if count3 == 0:
                    pages.append(np.nan)
                if count4 == 0:
                    binding.append(np.nan)

            except:
                saved=link
                print('dafuq')
                break

        else:
            saved=link
            print('You got caught!')
            break
    except:
        category.append(np.nan)
        ISBN.append(np.nan)
        pages.append(np.nan)
        binding.append(np.nan)
    

    time.sleep(random.uniform(0,3))

  0%|          | 0/13176 [00:00<?, ?it/s]

In [22]:
len(category),len(ISBN)

(13176, 13176)

In [23]:
len(pages),len(binding)

(13176, 13176)

In [24]:
df['Category'] = category
df['ISBN'] = ISBN
df['Pages'] = pages
df['Binding'] = binding

In [25]:
df.head()

,Title,Price_Old,Discount,Price_New,Publisher,Link,Year,Category,ISBN,Pages,Binding
0,22 Οκτωβρίου 1717: Η λησμονημένη…,10.60€,-11%,9.43€,Δημοσθένης Δόνος,https://www.captainbook.gr/book/22-oktobriou-1...,2017,Νεότερη & Σύγχρονη Ελληνική Ιστορία,9786188398603,72,Μαλακό εξώφυλλο
1,THE DUNE NOVELS MESSIAH,13.77€,-11%,12.26€,9781529347869,https://www.captainbook.gr/book/the-dune-novel...,2017,NaN,9781473655324,304,NaN
2,ΜΕΤΡΗΣΗ ΚΑΙ ΑΞΙΟΛΟΓΗΣΗ ΣΤΗΝ…,NaN,NaN,22.26€,ΚΩΝΣΤΑΝΤΙΝΟΣ ΠΑΠΑΝΑΣΤΑΣΙΟΥ,https://www.captainbook.gr/book/metrisi-kai-ak...,2017,NaN,9789963090358,NaN,NaN
3,Διεθνή Λογιστικά Πρότυπα,24.38€,-11%,21.70€,NaN,https://www.captainbook.gr/book/diethni-logist...,2017,NaN,9789609781183,430,Μαλακό εξώφυλλο
4,Χαράλαμπος Μ. Μουτσόπουλος,14.00€,-21%,11.06€,Κωνσταντίνος Δ. Κόλλιος,https://www.captainbook.gr/book/xaralampos-m-m...,2017,", Δοκίμια",9789608338845,328,Μαλακό εξώφυλλο


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13176 entries, 0 to 13175
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Title      13169 non-null  object
 1   Price_Old  10940 non-null  object
 2   Discount   10940 non-null  object
 3   Price_New  11438 non-null  object
 4   Publisher  11778 non-null  object
 5   Link       13170 non-null  object
 6   Year       13176 non-null  int64 
 7   Category   10081 non-null  object
 8   ISBN       13105 non-null  object
 9   Pages      9463 non-null   object
 10  Binding    9407 non-null   object
dtypes: int64(1), object(10)
memory usage: 1.1+ MB


In [27]:
df.to_csv('../data/books_2017.csv',index=False)

In [28]:
df = pd.read_csv('../data/books_2017.csv')

In [29]:
df.head(10)

,Title,Price_Old,Discount,Price_New,Publisher,Link,Year,Category,ISBN,Pages,Binding
0,22 Οκτωβρίου 1717: Η λησμονημένη…,10.60€,-11%,9.43€,Δημοσθένης Δόνος,https://www.captainbook.gr/book/22-oktobriou-1...,2017,Νεότερη & Σύγχρονη Ελληνική Ιστορία,9786188398603,72.0,Μαλακό εξώφυλλο
1,THE DUNE NOVELS MESSIAH,13.77€,-11%,12.26€,9781529347869,https://www.captainbook.gr/book/the-dune-novel...,2017,NaN,9781473655324,304.0,NaN
2,ΜΕΤΡΗΣΗ ΚΑΙ ΑΞΙΟΛΟΓΗΣΗ ΣΤΗΝ…,NaN,NaN,22.26€,ΚΩΝΣΤΑΝΤΙΝΟΣ ΠΑΠΑΝΑΣΤΑΣΙΟΥ,https://www.captainbook.gr/book/metrisi-kai-ak...,2017,NaN,9789963090358,NaN,NaN
3,Διεθνή Λογιστικά Πρότυπα,24.38€,-11%,21.70€,NaN,https://www.captainbook.gr/book/diethni-logist...,2017,NaN,9789609781183,430.0,Μαλακό εξώφυλλο
4,Χαράλαμπος Μ. Μουτσόπουλος,14.00€,-21%,11.06€,Κωνσταντίνος Δ. Κόλλιος,https://www.captainbook.gr/book/xaralampos-m-m...,2017,", Δοκίμια",9789608338845,328.0,Μαλακό εξώφυλλο
5,The Rose and the Dagger : The…,14.29€,-11%,12.72€,Renee Ahdieh,https://www.captainbook.gr/book/the-rose-and-t...,2017,NaN,9781473657960,432.0,NaN
6,Διοικητική Λογιστική,90.00€,-11%,80.10€,Belverd Needles,https://www.captainbook.gr/book/dioikitiki-log...,2017,Πολιτική | Οικονομία,9789963274444,712.0,Χαρτόδετο
7,Διοίκηση Χρηματοπιστωτικών…,90.00€,-11%,80.10€,Cornett Marcia M.,https://www.captainbook.gr/book/dioikisi-xrima...,2017,Πολιτική | Οικονομία,9789963274383,1000.0,Χαρτόδετο
8,Κασετίνα Χάρι Πότερ 1-7,12.00€,-11%,10.68€,J. K. Rowling,https://www.captainbook.gr/book/kasetina-xari-...,2017,", Εφηβική Λογοτεχνία",5213005510480,NaN,NaN
9,"ΘΕΜΑΤΑ & ΛΥΣΕΙΣ Τράπεζας Θεμάτων,…",10.60€,-11%,9.43€,Συλλογικό έργο,https://www.captainbook.gr/book/themata--and--...,2017,NaN,9789607016850,NaN,NaN


In [32]:
df.dropna(thresh=7).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12889 entries, 0 to 13169
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Title      12888 non-null  object 
 1   Price_Old  10939 non-null  object 
 2   Discount   10939 non-null  object 
 3   Price_New  11381 non-null  object 
 4   Publisher  11629 non-null  object 
 5   Link       12889 non-null  object 
 6   Year       12889 non-null  int64  
 7   Category   9987 non-null   object 
 8   ISBN       12826 non-null  object 
 9   Pages      9457 non-null   float64
 10  Binding    9388 non-null   object 
dtypes: float64(1), int64(1), object(9)
memory usage: 1.2+ MB
